# Experiment: Calculating Euclidean distance quantumly
This program is done for testing the accuracy of the Euclidean distance calculated by quantum hardware.

In [1]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.primitives import (StatevectorSampler, Estimator, StatevectorEstimator)
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
from qiskit.compiler import transpile
from qiskit_aer import AerSimulator
""" Create an account in IBM Quantum Platform https://quantum.ibm.com/ to obtain your own token"""
QiskitRuntimeService.save_account(channel="ibm_quantum", 
                                  token="<YOUR TOKEN>",
                                  set_as_default=True, overwrite=True)
service = QiskitRuntimeService()

In [2]:
import numpy as np
import math
import random

from sklearn.datasets import load_iris
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans

In [3]:
def classical_distance(point_1, point_2):
    """
    Classically calculate the Euclidean distance between 2 points
    """
    euclidean_distance = math.sqrt((point_1[0] - point_2[0]) ** 2 + (point_1[1] - point_2[1]) ** 2)
    return euclidean_distance

In [4]:
def polar_angle(point):
    """
    Calculating polar angle
    """
    polar_angle = math.atan2(point[1], point[0])
    return polar_angle

def polar_radius(point):
    """
    Calculating polar coordinate
    """
    polar_radius = math.sqrt(point[0] ** 2 + point[1] ** 2)
    return polar_radius

In [5]:
def quantum_distance(dataset, num_qubits):
    # Calculate the number of distances can be calculated at the same time
    num_distances = 0
    num_distances, remainder = divmod(num_qubits, 2)
    # Initialize quantum circuit
    qr = QuantumRegister(num_distances * 2, name="q")
    cr = ClassicalRegister(num_distances, name="c")
    qc = QuantumCircuit(qr, cr)
     # Initialize a the list contaisn all the pairs of points whose distances have been calculated
    calculated_distances = {}
    i = 0
    dataset_copy = dataset.copy()
    while len(dataset_copy) > 0:
        point1 = dataset_copy.pop()
        point2 = dataset_copy.pop()
        angle1 = polar_angle(point1)
        angle2 = polar_angle(point2)
        # Prepare quantum circuit
        qc.h(qr[i * 2])
        qc.cx(qr[i * 2], qr[i * 2 + 1])
        qc.ry(-abs(angle1 - angle2), qr[i * 2 + 1])
        qc.cx(qr[i * 2], qr[i * 2 + 1])
        qc.ry(abs(angle1 - angle2), qr[i * 2 + 1])
        # Inteferernce and measurement
        qc.h(qr[i * 2])
        qc.measure(qr[i * 2], cr[i])
        # Update the index and calculated_distances
        calculated_distances[i] = {'pair': [point1, point2], 'count_1': 0}
        i += 1
    # Execute the quantum circuit
    """Uncomment the following lines of code to run on quantum system"""
    backend = service.get_backend('ibm_kyoto')
    pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
    circuit = pm.run(qc)
    sampler = Sampler(backend)
    """Uncomment the following lines of code to run on noisy quantum simulator"""
    #real_backend = service.backend("ibm_kyoto")
    #aer = AerSimulator.from_backend(real_backend)
    #pm = generate_preset_pass_manager(backend=aer, optimization_level=1)
    #circuit = pm.run(qc)
    #sampler = Sampler(backend=aer)
    """ Uncomment the following lines of code to run on ideal local simulator"""
    #sampler = StatevectorSampler()
    job = sampler.run([circuit], shots=8192)
    result = job.result()[0]
    count = result.data.c.get_counts()
    state_list = list(count.keys())
    for state in state_list:
        for idx, bit in enumerate(state):
            state_count = count.get(state, 0)
            if bit == '1':
                if (len(calculated_distances.items()) - idx - 1) >= 0:
                    calculated_distances[len(calculated_distances.items()) - idx - 1]['count_1'] += state_count
    for key, value in calculated_distances.items():
        probability_1 = value['count_1'] / 8192
        point1 = value['pair'][0]
        point2 = value['pair'][1]
        # Calculate the distance from the probability of |1>
        radius1 = polar_radius(point1)
        radius2 = polar_radius(point2)
        distance = math.sqrt(radius1 ** 2 + radius2 ** 2 - 2 * radius1 * radius2 * (1 - 2 * probability_1))
        # Compare the quantum result with classical result
        classical_dist = classical_distance(point1, point2)
        qc_difference = abs(distance - classical_dist)
        print(f"Quantum-Classical distance between {point1} and {point2} difference: {qc_difference}")

In [6]:
num_data = 10
# Import the Iris dataset
iris_data = load_iris()
features = iris_data.data
labels = iris_data.target
# Normalize the features
# Apply MinMaxScaler to map data onto (0, 1)
features = MinMaxScaler().fit_transform(features)
# Reduce the number of features
features = PCA(n_components=2).fit_transform(features)
# Select randomly num_data indices
random_indices = np.random.choice(len(features), size=num_data, replace=False)
# Extract features and corresponding labels using random_indices
testing_features = features[random_indices]
features_list = testing_features.tolist()
quantum_distance(features_list, num_data)

C:\Users\Admin\AppData\Roaming\Python\Python39\site-packages\qiskit_ibm_runtime\qiskit_runtime_service.py:879: UserWarning: Your current pending jobs are estimated to consume 649.5556236231932 quantum seconds, but you only have 511 quantum seconds left in your monthly quota; therefore, it is likely this job will be canceled
  warnings.warn(warning_message)


Quantum-Classical distance between [-0.6257804985628276, 0.05713354113434377] and [0.5110202145758135, -0.13257591538169197] difference: 0.13558280196173866
Quantum-Classical distance between [-0.656537789993852, 0.010724491106869776] and [0.5139326307785076, 0.09888163229704708] difference: 0.02514720223568645
Quantum-Classical distance between [0.10383104269125022, -0.12178174239523801] and [0.5434892461418294, -0.054439910368968294] difference: 0.002121316530867723
Quantum-Classical distance between [0.48482666253145734, 0.11534865817996512] and [-0.06876975010843102, -0.18564800737703896] difference: 0.0070803077580446905
Quantum-Classical distance between [0.21948548888608416, 0.10938392765823472] and [0.36914899749077357, 0.06434807195275628] difference: 0.046844173753792545
